In [3]:
import seaborn as sns
import folium
import geopandas as gpd
import pandas as pd
from logging import warning

from utils.downloads import (
    Censo,
    Nivel,
    download_malha
)
from utils.geo import calc_similarity

In [4]:
sns.set_theme(rc={'figure.figsize':(11.7,8.27)})

# Métricas de avaliação, acurácia e F-score

Para avaliar a nossa metodologia, podemos utilizar métricas comummente utilizadas na avaliação de modelos de aprendizagem de máquina.

Na aprendizagem de máquina, a avaliação de modelos é uma etapa crucial para avaliar o desempenho de um modelo treinado em dados não vistos. Isso ajuda a entender o quão bem o modelo generaliza para novas instâncias e se está fazendo previsões precisas.

## Acurácia, F1-Score e conjuntos de dados desbalanceados

**Acurácia** é uma das métricas mais comumente usadas para avaliar modelos de classificação. Ela mede a proporção de instâncias classificadas corretamente em relação ao total de instâncias. A fórmula para acurácia é:

$$ \text{Acurácia} = \frac{\text{Número de Previsões Corretas}}{\text{Número Total de Previsões}} $$

No entanto, a acurácia por si só pode ser enganosa, especialmente ao lidar com conjuntos de dados desbalanceados, onde uma classe supera significativamente as outras. Nesses casos, um classificador pode alcançar alta acurácia simplesmente prevendo a classe majoritária para todas as instâncias, enquanto ignora completamente a classe minoritária. Isso pode levar a uma avaliação equivocada do desempenho do modelo.

O **F1-Score**, por outro lado, fornece uma medida melhor da precisão de um modelo ao considerar tanto a precisão quanto o recall. É a média harmônica da precisão e do recall, dando peso igual a ambas as métricas. A fórmula para o F1-Score é:

$$ F1 = 2 \times \frac{\text{Precisão} \times \text{Recall}}{\text{Precisão} + \text{Recall}} $$

Onde:

- Precisão mede a proporção de previsões verdadeiras positivas em relação a todas as previsões positivas, e é definida como:

$$ \text{Precisão} = \frac{\text{Verdadeiros Positivos}}{\text{Verdadeiros Positivos} + \text{Falsos Positivos}} $$

- Recall (também conhecido como sensibilidade) mede a proporção de previsões verdadeiras positivas em relação a todas as instâncias positivas reais, e é definido como:

$$ \text{Recall} = \frac{\text{Verdadeiros Positivos}}{\text{Verdadeiros Positivos} + \text{Falsos Negativos}} $$

O F1-Score considera tanto falsos positivos quanto falsos negativos, tornando-o uma métrica mais confiável para avaliar modelos em conjuntos de dados desbalanceados. Ele equilibra o compromisso entre precisão e recall, fornecendo uma única pontuação que reflete o desempenho do modelo em ambas as classes.


Adaptando ao nosso contexto, visto que a base de dados contendo os dados verdadeiros contém apenas os casos onde o relacionamento entre dois setores censitários é verdadeiro, podemos considerá-la uma base de registros positivos, e todas as combinações de setores não presentes nessa base como registros negativos.

Assim, tratando cada combinação possível entre setores censistários de 2010 e 2022 como nossa unidade, podemos avaliar nossa metodologia como um modelo de classificação de uma categoria.

# Carregando os dados

Primeiro, vamos carregar os dados básicos dos setores censitários de 2022 e os dados de similaridade gerados nos notebooks anteriores.

In [5]:
%%time
setores10 = download_malha(Censo.CENSO_2010, Nivel.SETORES)
setores10 = setores10[setores10['CD_GEOCODM'] == '3550308']
setores10 = setores10.to_crs(epsg=31983)
setores10.sample(3)

CPU times: user 18.7 s, sys: 93.4 ms, total: 18.8 s
Wall time: 19 s


,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
46090,111602.0,355030819000006,URBANO,35503081900,None,355030819,CAPÃO REDONDO,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,46091,"POLYGON ((319349.692 7381997.221, 319348.350 7..."
61263,126845.0,355030891000126,URBANO,35503089100,None,355030891,VILA MATILDE,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,61264,"POLYGON ((342804.621 7395800.681, 342805.817 7..."
52304,117886.0,355030844000064,RURAL,35503084400,None,355030844,JARDIM HELENA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,52305,"POLYGON ((356404.355 7402003.428, 356419.933 7..."


In [6]:
%%time
setores22 = download_malha(Censo.CENSO_2022, Nivel.SETORES)
setores22 = setores22[setores22['CD_MUN'] == '3550308']
setores22 = setores22.to_crs(epsg=31983)
setores22.sample(3)

CPU times: user 1min 5s, sys: 455 ms, total: 1min 6s
Wall time: 1min 6s


,CD_SETOR,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,...,CD_CONCURB,NM_CONCURB,v0001,v0002,v0003,v0004,v0005,v0006,v0007,geometry
77202,355030838000709P,0.111574,3,Sudeste,35,São Paulo,3550308,São Paulo,355030838,Jabaquara,...,3550308,São Paulo/SP,0,0,0,0,0.000000,0.000000,0,"POLYGON ((333727.517 7385627.529, 333729.550 7..."
77137,355030838000644P,0.005954,3,Sudeste,35,São Paulo,3550308,São Paulo,355030838,Jabaquara,...,3550308,São Paulo/SP,263,156,156,0,1.977444,0.751880,133,"POLYGON ((332021.466 7383311.801, 332009.679 7..."
68628,355030815000273P,0.071433,3,Sudeste,35,São Paulo,3550308,São Paulo,355030815,Campo Belo,...,3550308,São Paulo/SP,196,81,81,0,2.684932,12.328767,73,"POLYGON ((329414.448 7385882.210, 329410.477 7..."


In [7]:
sim_10_path = 'data/01_similarity_10.parquet'
sim_10 = gpd.read_parquet(sim_10_path)
sim_10['comp_id'] = sim_10['CD_GEOCODI'] + '-' + sim_10['CD_SETOR']
sim_10

,CD_SETOR,CD_GEOCODI,geometry,inter_area,inter_perc,comp_id
0,355030801000001P,355030801000001,"POLYGON ((339811.471 7392866.307, 339812.545 7...",70930.590833,9.833295e-01,355030801000001-355030801000001P
1,355030801000001P,355030801000006,"MULTIPOLYGON (((339784.892 7392592.542, 339819...",373.279509,3.782129e-03,355030801000006-355030801000001P
2,355030801000001P,355030801000010,"POLYGON ((339456.132 7392739.111, 339531.109 7...",9.240690,1.543379e-04,355030801000010-355030801000001P
3,355030801000001P,355030801000011,"POLYGON ((339661.597 7392807.484, 339808.953 7...",471.770933,8.274765e-03,355030801000011-355030801000001P
4,355030801000002P,355030801000001,"POLYGON ((339810.127 7392984.872, 339814.260 7...",18.451154,2.557932e-04,355030801000001-355030801000002P
...,...,...,...,...,...,...
114396,355030896000338P,355030896000104,"POLYGON ((355815.970 7397540.784, 355573.189 7...",33668.997801,7.866150e-01,355030896000104-355030896000338P
114397,355030896000339P,355030884000164,"POLYGON ((355931.532 7397554.886, 355953.783 7...",0.003980,1.136118e-07,355030884000164-355030896000339P
114398,355030896000339P,355030896000044,"MULTIPOLYGON (((355937.670 7397481.306, 355952...",2.775304,4.792580e-05,355030896000044-355030896000339P
114399,355030896000339P,355030896000103,"MULTIPOLYGON (((355825.873 7397538.347, 355877...",6952.030517,1.608160e-01,355030896000103-355030896000339P


In [8]:
sim_viario_10_path = 'data/02_similarity_viario_10.parquet'
sim_viario_10 = gpd.read_parquet(sim_viario_10_path)
sim_viario_10['comp_id'] = sim_viario_10['CD_GEOCODI'] + '-' + sim_viario_10['CD_SETOR']
sim_viario_10

,CD_SETOR,CD_GEOCODI,geometry,inter_area,inter_perc,comp_id
0,355030801000001P,355030801000001,"MULTIPOLYGON (((339806.356 7392596.796, 339779...",60142.847118,9.960823e-01,355030801000001-355030801000001P
1,355030801000001P,355030801000006,"POLYGON ((339708.368 7392605.968, 339707.507 7...",47.562187,6.429313e-04,355030801000006-355030801000001P
2,355030801000002P,355030801000002,"MULTIPOLYGON (((339956.877 7392696.391, 339946...",61038.586717,9.960391e-01,355030801000002-355030801000002P
3,355030801000002P,355030801000004,"MULTIPOLYGON (((339897.400 7392679.461, 339888...",504.763397,8.958203e-03,355030801000004-355030801000002P
4,355030801000003P,355030801000002,"MULTIPOLYGON (((339915.684 7392696.058, 339916...",242.683018,3.960147e-03,355030801000002-355030801000003P
...,...,...,...,...,...,...
87836,355030896000338P,355030896000103,"MULTIPOLYGON (((355571.342 7397652.973, 355572...",122.522044,3.361404e-03,355030896000103-355030896000338P
87837,355030896000338P,355030896000104,"MULTIPOLYGON (((355712.152 7397608.612, 355714...",26629.910122,7.573375e-01,355030896000104-355030896000338P
87838,355030896000339P,355030896000044,"POLYGON ((355967.757 7397511.604, 355968.423 7...",0.000118,2.325016e-09,355030896000044-355030896000339P
87839,355030896000339P,355030896000103,"MULTIPOLYGON (((355805.446 7397532.150, 355816...",5789.638008,1.588393e-01,355030896000103-355030896000339P


In [9]:
sim_double_buff_10_path = 'data/06_similarity_double_buffer_10.parquet'
sim_double_buff_10 = gpd.read_parquet(sim_double_buff_10_path)
sim_double_buff_10['comp_id'] = sim_double_buff_10['CD_GEOCODI'] + '-' + sim_double_buff_10['CD_SETOR']
sim_double_buff_10

,CD_GEOCODI,CD_SETOR,geometry,inter_area,inter_perc,comp_id
66320,355030804000079,355030804000079P,"MULTIPOLYGON (((345004.253 7391489.517, 345004...",27868.809593,1.000000,355030804000079-355030804000079P
66321,355030804000080,355030804000080P,"MULTIPOLYGON (((344747.028 7391454.176, 344885...",29099.528367,1.000000,355030804000080-355030804000080P
66322,355030804000081,355030804000081P,"MULTIPOLYGON (((344686.207 7391268.311, 344688...",17257.656772,1.000000,355030804000081-355030804000081P
66321,355030804000082,355030804000080P,"POLYGON ((344743.884 7391449.027, 344742.477 7...",3.674850,0.000137,355030804000082-355030804000080P
66323,355030804000082,355030804000082P,"MULTIPOLYGON (((344737.762 7391446.652, 344737...",26889.176795,0.999863,355030804000082-355030804000082P
...,...,...,...,...,...,...
93242,355030896000246,355030896000227P,"POLYGON ((354771.973 7397966.378, 354814.526 7...",147.748258,0.025620,355030896000246-355030896000227P
93260,355030896000246,355030896000245P,"POLYGON ((354771.670 7397970.756, 354750.848 7...",1669.246568,0.289455,355030896000246-355030896000245P
93261,355030896000246,355030896000246P,"POLYGON ((354784.574 7397992.486, 354788.166 7...",3949.865171,0.684925,355030896000246-355030896000246P
93212,355030896000247,355030896000193P,"POLYGON ((357430.206 7395524.645, 357430.202 7...",980.495123,0.096088,355030896000247-355030896000193P


In [10]:
sim_double_buff_resga_10_path = 'data/06_similarity_double_buffer_resga_10.parquet'
sim_double_buff_resga_10 = gpd.read_parquet(sim_double_buff_resga_10_path)
sim_double_buff_resga_10['comp_id'] = sim_double_buff_resga_10['CD_GEOCODI'] + '-' + sim_double_buff_resga_10['CD_SETOR']
sim_double_buff_resga_10

,CD_GEOCODI,CD_SETOR,geometry,inter_area,inter_perc,comp_id
557,355030804000079,355030804000079P,"MULTIPOLYGON (((344923.185 7391231.361, 344925...",27868.809593,1.000000,355030804000079-355030804000079P
558,355030804000080,355030804000080P,"MULTIPOLYGON (((344852.331 7391245.510, 344855...",29099.528367,1.000000,355030804000080-355030804000080P
559,355030804000081,355030804000081P,"MULTIPOLYGON (((344688.119 7391274.180, 344692...",17257.656772,1.000000,355030804000081-355030804000081P
558,355030804000082,355030804000080P,"POLYGON ((344742.196 7391443.694, 344742.081 7...",3.674850,0.000137,355030804000082-355030804000080P
560,355030804000082,355030804000082P,"MULTIPOLYGON (((344643.686 7391295.269, 344650...",26889.176795,0.999863,355030804000082-355030804000082P
...,...,...,...,...,...,...
27476,355030896000246,355030896000227P,"POLYGON ((354771.973 7397966.378, 354814.526 7...",147.748258,0.025620,355030896000246-355030896000227P
27494,355030896000246,355030896000245P,"POLYGON ((354750.848 7397978.475, 354780.511 7...",1669.246568,0.289455,355030896000246-355030896000245P
27495,355030896000246,355030896000246P,"POLYGON ((354784.574 7397992.486, 354788.166 7...",3949.865171,0.684925,355030896000246-355030896000246P
27446,355030896000247,355030896000193P,"POLYGON ((357430.206 7395524.645, 357430.202 7...",980.495123,0.096088,355030896000247-355030896000193P


In [11]:
%%time
setores10 = download_malha(Censo.CENSO_2010, Nivel.SETORES)
setores10 = setores10[setores10['CD_GEOCODM'] == '3550308']
setores10 = setores10.to_crs(epsg=31983)
setores10.sample(3)

CPU times: user 17.9 s, sys: 607 ms, total: 18.5 s
Wall time: 18.8 s


,ID,CD_GEOCODI,TIPO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,CD_GEOCODB,NM_BAIRRO,ID1,geometry
47170,112738.0,355030823000096,URBANO,35503082300,None,355030823,CIDADE DUTRA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,47171,"POLYGON ((326297.494 7376657.395, 326335.534 7..."
58765,124290.0,355030876000328,URBANO,35503087600,None,355030876,SAPOPEMBA,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,58766,"POLYGON ((345695.452 7389501.498, 345679.621 7..."
54342,119851.0,355030855000079,URBANO,35503085500,None,355030855,PARELHEIROS,3550308,SÃO PAULO,SÃO PAULO,METROPOLITANA DE SÃO PAULO,None,None,54343,"POLYGON ((326264.438 7360189.961, 326245.336 7..."


In [12]:
comparabilidade_ibge = pd.read_excel('data/Comparabilidade - Setores Censitários 2022 - 2010.xlsx',
                                     dtype={'SETOR_2010': str, 'SETOR_2022': str})
comparabilidade_ibge = comparabilidade_ibge[comparabilidade_ibge['SETOR_2010'].str.startswith('3550308')]
comparabilidade_ibge['SETOR_2022'] = comparabilidade_ibge['SETOR_2022'] + 'P'
comparabilidade_ibge['comp_id'] = comparabilidade_ibge['SETOR_2010'] + '-' + comparabilidade_ibge['SETOR_2022']
comparabilidade_ibge

,SETOR_2022,SETOR_2010,comp_id
42455,355030812000159P,355030812000068,355030812000068-355030812000159P
42456,355030814000066P,355030814000002,355030814000002-355030814000066P
42457,355030815000282P,355030815000081,355030815000081-355030815000282P
42458,355030815000294P,355030815000104,355030815000104-355030815000294P
42459,355030815000295P,355030815000113,355030815000113-355030815000295P
...,...,...,...
449495,355030894000308P,355030894000192,355030894000192-355030894000308P
449496,355030894000310P,355030894000114,355030894000114-355030894000310P
449497,355030894000315P,355030894000260,355030894000260-355030894000315P
449498,355030896000058P,355030896000058,355030896000058-355030896000058P


# Observações iniciais

Apesar de ser nossa única fonte vinda do IBGE e, portanto, nossa melhor alternativa como *source of truth*, a planilha com o mapemento entre os setores dos censos de 2010 e 2022 apresenta alguns casos estranhos.

Abaixo, deixo um caso para a comparação entre o mapamento presente na planilha e o mapeamento obtido pela nossa metodologia.

Primeiro, o mapeamento presente na tabela fornecida pelo IBGE.

In [13]:
setor_exemplo = '355030866000077'

In [14]:
comparabilidade_ibge.query(f'SETOR_2010 == "{setor_exemplo}"')

,SETOR_2022,SETOR_2010,comp_id
274988,355030866000228P,355030866000077,355030866000077-355030866000228P
274989,355030866000229P,355030866000077,355030866000077-355030866000229P
332550,355030866000227P,355030866000077,355030866000077-355030866000227P


In [15]:
setores_exemplo_22 = comparabilidade_ibge.loc[comparabilidade_ibge['SETOR_2010']==setor_exemplo, 'SETOR_2022']

exemplo_map_1 = setores22.query('CD_SETOR in @setores_exemplo_22').explore(
    tiles='CartoDB positron',
    name='Setores (2022)',  # name of the layer in the map
    color='royalblue',
    style_kwds={'weight': 1},
)

exemplo_map_1 = setores10.query(f'CD_GEOCODI == "{setor_exemplo}"').explore(
    m=exemplo_map_1,
    tiles='CartoDB positron',
    name='Setores (2010)',  # name of the layer in the map
    color='darkred',
    style_kwds={'weight': 1},
)

folium.LayerControl().add_to(exemplo_map_1)  # use folium to add layer control

exemplo_map_1

Agora, o mapeamento obtido pela nossa metodologia.

In [16]:
calc_similarity(setores10.query(f'CD_GEOCODI == "{setor_exemplo}"'), setores22, 'CD_GEOCODI', 'CD_SETOR')

,CD_GEOCODI,CD_SETOR,geometry,inter_area,inter_perc
84775,355030866000077,355030866000076P,"POLYGON ((332590.713 7394340.276, 332589.967 7...",596.143657,0.226384
84896,355030866000077,355030866000228P,"POLYGON ((332590.713 7394340.276, 332552.254 7...",701.500714,0.266393
84897,355030866000077,355030866000229P,"POLYGON ((332589.967 7394324.836, 332589.406 7...",614.780992,0.233461
84948,355030866000077,355030866000284P,"POLYGON ((332589.967 7394324.836, 332590.713 7...",720.906097,0.273762


In [17]:
setores_exemplo_22 = calc_similarity(setores10.query(f'CD_GEOCODI == "{setor_exemplo}"'), setores22, 'CD_GEOCODI', 'CD_SETOR')['CD_SETOR']

exemplo_map_2 = setores22.query('CD_SETOR in @setores_exemplo_22').explore(
    tiles='CartoDB positron',
    name='Setores (2022)',  # name of the layer in the map
    color='royalblue',
    style_kwds={'weight': 1},
)

exemplo_map_2 = setores10.query(f'CD_GEOCODI == "{setor_exemplo}"').explore(
    m=exemplo_map_2,
    tiles='CartoDB positron',
    name='Setores (2010)',  # name of the layer in the map
    color='darkred',
    style_kwds={'weight': 1},
)

folium.LayerControl().add_to(exemplo_map_2)  # use folium to add layer control

exemplo_map_2

Apesar de ser uma valiação difícil sem maiores informações, puramente pela visualização no mapa, nossa metodologia parece mais adequada do que os dados presentes na tabela.

De todo modo, seguiremos utilizando a tabela fornecida pelo IBGE como fonte dos dados verdadeiros.

# Avaliando os modelos

In [18]:
def model_eval(pred:pd.DataFrame|gpd.GeoDataFrame, true:pd.DataFrame|gpd.GeoDataFrame, total_predictions:int) -> tuple[int, int, int, int]:
    right_non_zero = true.merge(pred, how='inner')
    true_positive = right_non_zero.shape[0]
    false_negative = true.shape[0]-true_positive
    false_positive = pred.shape[0]-true_positive
    true_negative = total_predictions - true_positive - false_negative - false_positive
    return (true_positive, true_negative, false_positive, false_negative)

In [19]:
def f1(true_positive:int, true_negative:int, false_positive:int, false_negative:int) -> float:
    prec = true_positive/(true_positive+false_positive)
    rec = true_positive/(true_positive+false_negative)
    return (2*prec*rec)/(prec+rec)

In [20]:
def tpr(true_positive:int, true_negative:int, false_positive:int, false_negative:int) -> float:
    return true_positive/(true_positive+false_negative)

## Análise geográfica simples

In [21]:
sim_10_eval = model_eval(sim_10, comparabilidade_ibge, setores10.shape[0]*setores22.shape[0])
sim_10_eval

(27242, 522836118, 87159, 657)

In [22]:
f1(*sim_10_eval)

0.3828812368236121

In [23]:
tpr(*sim_10_eval)

0.9764507688447615


No Modelo Base, observamos um número relativamente alto de falsos positivos (87159), o que indica instâncias em que o modelo previu incorretamente um resultado positivo quando o resultado real era negativo. Isso sugere que o modelo tem uma tendência a superestimar casos positivos, levando a uma menor precisão e, consequentemente, contribuindo para o F1-Score mais baixo.

Isso era o esperado, visto que este modelo contém apenas o calculo de similaridade entre as geometrias, sem nenhum tratamento prévio. Esse modelo é importante, porém, para servir como base de comparação com os demais modelos.

## Remoção do viário

In [20]:
sim_viario_10_eval = model_eval(sim_viario_10, comparabilidade_ibge, setores10.shape[0]*setores22.shape[0])
sim_viario_10_eval

(27226, 522862662, 60615, 673)

In [21]:
f1(*sim_viario_10_eval)

0.4704682909970624

In [ ]:
tpr(*sim_viario_10_eval)


Comparado ao Modelo Base, o modelo com remoção do viário mostra uma leve melhora no F1-Score. Enquanto os verdadeiros positivos e negativos são semelhantes, há uma redução nos falsos positivos (60615), indicando que este modelo tem uma melhor capacidade de distinguir entre casos positivos e negativos, resultando em uma maior precisão.

## Buffers negativo nos setores e positivo na malha viária

In [22]:
sim_double_buff_10_eval = model_eval(sim_double_buff_10, comparabilidade_ibge, setores10.shape[0]*setores22.shape[0])
sim_double_buff_10_eval

(27188, 522905279, 17998, 711)

In [23]:
f1(*sim_double_buff_10_eval)

0.7440103988506535

In [ ]:
tpr(*sim_double_buff_10_eval)


O modelo com a adição de buffers demonstra uma diminuição significativa nos falsos positivos (17998) em comparação com os modelos anteriores. Isso indica uma melhoria substancial na capacidade do modelo de classificar corretamente os casos negativos, resultando em uma maior precisão e melhor desempenho geral, como refletido no F1-Score mais alto.

## Buffers e remoção de resgas

In [24]:
sim_double_buff_resga_10_eval = model_eval(sim_double_buff_resga_10, comparabilidade_ibge, setores10.shape[0]*setores22.shape[0])
sim_double_buff_resga_10_eval

(27188, 522905418, 17859, 711)

In [25]:
f1(*sim_double_buff_resga_10_eval)

0.7454281249143202

In [26]:
tpr(*sim_double_buff_resga_10_eval)

0.9745152155991255

Similar ao modelo anterior, o modelo com buffers e remoção de resgas mantém um baixo número de falsos positivos (17859). A adição da remoção de resgas, entretanto, não impacta significativamente os falsos positivos em comparação com o modelo anterior, indicando que a melhoria no desempenho é atribuída principalmente à aplicação dos buffers.

## Resumo


Como era esperado, os modelos com a aplicação de buffers teve performance bastante superior aos modelos anteriores, devido ao maior cuidado com os dados utilizados para o cálculo de similaridade.

A remoção de resgas, no entanto, não demonstrou melhora tão significativa na avaliação do modelo, de modo que vale considerar se adicionamos essa complexidade adicional ao modelo.